## МОДУЛЬ ВИЗУАЛИЗАЦИИ УГЛОВ ЭЙЛЕРА
---
МОДУЛЬ ПРЕДНАЗНАЧЕН ДЛЯ ВИЗУАЛИИЗАЦИИ ПОКАЗАНИЙ УГЛОВ ЭЙЛЕРА ПОЛУЧЕННЫХ ПО КАНАЛУ ГИРОСКОПА, АКСЕЛЕРОМЕТРА, КОМПЛИМЕНТАРНОГО ФИЛЬТРА

### Блок импорта библиотек необходимых для работы модуля

In [ ]:
# Подключить библиотеки для мат обработки данных и генерации случайных данных
import numpy as np
import math

# Подключить библиотеки для работы с датой/временем
from datetime import datetime
import time

# Подключить библиотеки для асинхронной работы
import asyncio
# Подключить библиотеки для работы с базой данных redis
import redis
# Подключить библиотеки для работы с JSON
import json

# Подключить библиотеки для работы с виджетами в Jupyter Lab
import ipywidgets as widgets
from   ipywidgets import Output
from IPython.display import display

# -----------------------------------------------------------------------
# Подключить пользовательские модули
# -----------------------------------------------------------------------
import sys
sys.path.append('/home/project/3D-position-2023/py/lib')

# Подключить модуль для подключения к БД Redis
from py.lib.lib_work_redis_rev01_v01 import *
# Подключить модуль для создания и обновления HTML элементов отображения данных об углах Эйлера
from py.lib.lib_work_html_widget_rev01_v01 import *
# Подключить модуль с оберткой для синхронных функций в стиле setInterval JavaScript
from py.lib.lib_setinterval_rev01_v01 import *

### Блок настроек среды исполнения Jupyter lab

In [ ]:
# Инструкция требующая повторно перезагружать пользовательские функции после изменения их кода
%load_ext autoreload
%autoreload 2

### Блок объявления глобальных переменных модуля

In [ ]:
# Создать алиасы индексов соответствующих под-массивов, для удобства обращения
ROLL_IND    = 0
PITCH_IND   = 1
YAW_IND     = 2

AccArrRaw  = [[0,0,0],[0,0,0],[0,0,0]] # массив, хранит "сырые" данные от Акселерометра
GyroArrRaw = [[0,0,0],[0,0,0],[0,0,0]] # массив, хранит "сырые" данные от Гироскопа
MagArrRaw  = [[0,0,0],[0,0,0],[0,0,0]] # массив, хранит "сырые" данные от Магнитометра

AccArrAvg  = [[0,0,0],[0,0,0],[0,0,0]] # массив, хранит усредненные данные от Акселерометра
GyroArrAvg = [[0,0,0],[0,0,0],[0,0,0]] # массив, хранит усредненные данные от Гироскопа
MagArrAvg  = [[0,0,0],[0,0,0],[0,0,0]] # массив, хранит усредненные данные от Магнитометра

AccArrAngle  = [[0,0,0],[0,0,0],[0,0,0]] # массив, хранит углы Эйлера полученные с от Акселерометра
GyroArrAngle = [[0,0,0],[0,0,0],[0,0,0]] # массив, хранит углы Эйлера полученные с помощью Гироскопа
CompArrAngle = [[0,0,0],[0,0,0],[0,0,0]] # массив, хранит углы Эйлера полученные с помощью фильтра слияния

TempArr = [0] # массив, хранит данные от термометра

TimeDeltaAngle = 0 # значение dt для текущей итерации вычисления углов Эйлера

Alpha = 0.98 # коэффициент фильтра слияния

In [ ]:
ConnectDB =  ConnectDB() # выполнить подключение к базе данных Redis

### Блок синхронизации данных об углах Эйлера с базой данных

In [2]:
'''
        Не системная функция, выполняет синхронизацию данных между БД и переменными блокнота.  
'''
def AsyncReadDataDB() -> None:
        # Считать массив "сырых" данных из БД
        ImuListRawKey = ('AccArrRaw', 'GyroArrRaw', 'MagArrRaw')
        ImuListRawData = [AccArrRaw, GyroArrRaw, MagArrRaw]
        ReadValJSONfromDB( ConnectDB, ImuListRawKey, ImuListRawData )

        # Считать массив  усредненных данных из БД
        ImuListAvgKey = ('AccArrAvg', 'GyroArrAvg', 'MagArrAvg')
        ImuListAvgData = [AccArrAvg, GyroArrAvg, MagArrAvg]
        ReadValJSONfromDB( ConnectDB, ImuListAvgKey, ImuListAvgData )

        # Считать массивы со значениями углов Эйлера из БД
        AngelListKey = ('AccArrAngle', 'GyroArrAngle', 'CompArrAngle')
        AngelListData = [AccArrAngle, GyroArrAngle, CompArrAngle]
        ReadValJSONfromDB( ConnectDB, AngelListKey, AngelListData )

        # Считать массив со значениями температуры IMU из БД
        TempListKey = ('TempArr',)
        TempListData = [TempArr]
        ReadValJSONfromDB( ConnectDB, TempListKey, TempListData )   

        # Считать значение дельты времени обращений к IMU из БД
        TimeDeltaAngleKey = ('TimeDeltaAngle',)
        TimeDeltaAngleData = [TimeDeltaAngle]
        ReadValJSONfromDB( ConnectDB, TimeDeltaAngleKey, TimeDeltaAngleData )

        # Считать значение коэффициента Альфа, используемого при вычисления значений углов Эйлера в фильтре слияния
        AlphaKey = ('AlphaKey',)
        AlphaData = [Alpha]
        ReadValJSONfromDB( ConnectDB, AlphaKey, AlphaData )

In [ ]:
# Асинхронная функция-обертка для выполнения стороннего кода в асинхронном режиме
async def AsyncDataDB():
    AsyncFunc = SetInterval( 0.25, AsyncReadDataDB ) # период синхронизации 'ms' и 'fn' чтения данных из БД
    AsyncFunc.start()

await AsyncDataDB()

### Блок визуализации углов Эйлера по каналу Акселерометра

In [ ]:
# Создать виджеты для отображения значений углов Эйлера по каналу Акселерометра
Roll_Acc, Pitch_Acc, Yaw_Acc, A_Acc, B_Acc = CreateHTMLwidgest()
# Отобразить виджеты
display(widgets.HBox([widgets.Label(value='Roll_Acc'),\
                     Roll_Acc,\
                     A_Acc,\
                     widgets.Label(value='Pitch_Acc'),\
                     Pitch_Acc,\
                     B_Acc,\
                     widgets.Label(value='Yaw_Acc'),\
                     Yaw_Acc]))

# Создать асинхронную обертку кода для асинхронного выполнения кода функции обновления содержимого виджетов углов по каналу Акселерометра
async def AsyncAccEulerUpdateVal():
    AsyncFunc = SetInterval( 0.25, UpdateHTMLwidgest, Roll_Acc, Pitch_Acc, Yaw_Acc, AccArrAngle ) # период обновления виджетов в 'ms' и 'fn'
    AsyncFunc.start()

await AsyncAccEulerUpdateVal()

### Блок визуализации углов Эйлера по каналу Гироскопа

In [ ]:
# Создать виджеты для отображения значений углов Эйлера по каналу Акселерометра
Roll_Gyro, Pitch_Gyro, Yaw_Gyro, A_Gyro, B_Gyro = CreateHTMLwidgest()
# Отобразить виджеты
display(widgets.HBox([widgets.Label(value='Roll_Gyr'),\
                     Roll_Gyro,\
                     A_Gyro,\
                     widgets.Label(value='Pitch_Gyr'),\
                     Pitch_Gyro,\
                     B_Gyro,\
                     widgets.Label(value='Yaw_Gyr'),\
                     Yaw_Gyro]))

# Создать асинхронную обертку кода для асинхронного выполнения кода функции обновления содержимого виджетов углов по каналу Акселерометра
async def AsyncAccEulerUpdateVal():
    AsyncFunc = SetInterval( 0.25, UpdateHTMLwidgest, Roll_Gyro, Pitch_Gyro, Yaw_Gyro, GyroArrAngle ) # период обновления виджетов в 'ms' и 'fn'
    AsyncFunc.start()

await AsyncAccEulerUpdateVal()

### Блок визуализации углов Эйлера по каналу Комплиментарного фильтра

In [ ]:
# Создать виджеты для отображения значений углов Эйлера по каналу Акселерометра
Roll_Com, Pitch_Com, Yaw_Com, A_com, B_Com = CreateHTMLwidgest()
# Отобразить виджеты
display(widgets.HBox([widgets.Label(value='Roll_Com'),\
                     Roll_Com,\
                     A_com,\
                     widgets.Label(value='Pitch_Com'),\
                     Pitch_Com,\
                     B_Com,\
                     widgets.Label(value='Yaw_Com'),\
                     Yaw_Com]))

# Создать асинхронную обертку кода для асинхронного выполнения кода функции обновления содержимого виджетов углов по каналу Акселерометра
async def AsyncAccEulerUpdateVal():
    AsyncFunc = SetInterval( 0.25, UpdateHTMLwidgest, Roll_Com, Pitch_Com, Yaw_Com, CompArrAngle ) # период обновления виджетов в 'ms' и 'fn'
    AsyncFunc.start()

await AsyncAccEulerUpdateVal()